## RAG Basics - Practice for lecture

In [ ]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0, 
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('../docs_for_rag/coolie_large.txt')

document = loader.load()

for document in loader.lazy_load():
    print(document)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

documents = text_splitter.split_documents(document)

# chunks = text_splitter.split_text(document[0].page_content)

##### OpenAI Embedding

In [ ]:
from langchain_openai import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings()

document_texts = [document.page_content for document in documents]

document_vectors = openai_embeddings.embed_documents(document_texts)

##### Google embedding

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

google_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

document_vectors = google_embeddings.embed_documents(document_texts)

print(document_vectors)

##### Ollama Embedding

In [ ]:
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(model="nomic-embed-text")

document_vectors = ollama_embeddings.embed_documents(document_texts)

##### Huggingface embedding

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

document_vectors = huggingface_embeddings.embed_documents(document_texts)

print(len(document_vectors[0]))
print(document_vectors[0][:10])

##### FAISS - Creating vectorstore from already created embeddings vs creating from documents

In [ ]:
from langchain_community.vectorstores import FAISS

# vectorstore = FAISS.from_documents(documents, openai_embeddings)
# vectorstore.similarity_search("who is dahaa?")


vectorstore = FAISS.from_embeddings(text_embeddings=list(zip(document_texts, document_vectors)), 
                                    embedding=openai_embeddings)
vectorstore.save_local('./faiss_vectorstore')

##### Loading a vector store from local disk storage

In [ ]:
loaded_vectorstore = FAISS.load_local(
    "./faiss_vectorstore", 
    openai_embeddings,  # Must use same embedding model that created it
    allow_dangerous_deserialization=True
)

##### Chaining

In [ ]:
prompt_template = PromptTemplate.from_template(
    """Answer my question only by using the below context.
    Context: {context}
    Question: {question}
    Answer:"""
)

vectorstore_for_chain = loaded_vectorstore.as_retriever()

chain = (
    {"context": vectorstore_for_chain, "question": RunnablePassthrough()}
    | prompt_template 
    | google_llm 
    | StrOutputParser()
)

chain.invoke("Who is Dahaa?")
